# Fake News Detection using LSTM and Deep learning 

In [ ]:
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install nltk
!pip install wordcloud


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import wordcloud as WordCloud
import re

# Exploring fake news

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
fake = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/Fake.csv')


In [ ]:
fake.head()


In [ ]:
fake.columns

In [ ]:
fake['subject'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='subject', data=fake)
  # Rotate the x-axis labels for better readability if needed
plt.show()

# Wordcloud

In [ ]:
text=''.join(fake['text'].tolist())

In [ ]:
' '.join(['this', 'is', 'a','data'])

In [ ]:
import wordcloud
import matplotlib.pyplot as plt


wordcloud = wordcloud.WordCloud(width=1920, height=1080).generate(text)
fig = plt.figure(figsize=(12, 6))

plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


# Exploring real news

In [ ]:
real = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/True.csv')

In [ ]:
text=''.join(real['text'].tolist())

In [ ]:
import wordcloud
import matplotlib.pyplot as plt


wordcloud = wordcloud.WordCloud(width=1920, height=1080).generate(text)
fig = plt.figure(figsize=(12, 6))

plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
real.sample(5)

In [ ]:
from numpy import record
unknown_publishers = []
for index, row in  enumerate(real.text.values):
  try:
    record = row.split('-', maxsplit=1)
    record[1]
    assert(len(record[0])<120)
  except:
    unknown_publishers.append(index)

In [ ]:
len(unknown_publishers)

In [ ]:
real.iloc[unknown_publishers].text

In [ ]:
real.iloc[8970]

In [ ]:
real= real.drop(8970, axis=0)

In [ ]:
from pandas.core.common import temp_setattr

publisher = []
tmp_text = []

for index, row in enumerate(real.text.values):
    if index in unknown_publishers:
        tmp_text.append(row)
        publisher.append('Unknown')
    else:
        record = row.split('-', maxsplit=1)
        publisher.append(record[0].strip())
        tmp_text.append(record[1].strip())


In [ ]:
real['publisher']=publisher
real['text'] = tmp_text

In [ ]:
real.head()

In [ ]:
real.shape

In [ ]:
empty_fake_index = [index for index, text in enumerate(fake.text.tolist()) if str(text).strip()==""]


In [ ]:
fake.iloc[empty_fake_index]

In [ ]:
real['text'] = real['title'] + " " + real['text']
fake['text'] = fake['title'] + " " + fake['text']

In [ ]:
real['text']= real['text'].apply(lambda x:str(x).lower())
fake['text']=fake['text'].apply(lambda x:str(x).lower())

# Preprocessing the text

In [ ]:
real['class']=1
fake['class']=0

In [ ]:
real.columns

In [ ]:
real= real[['text', 'class']]

In [ ]:
fake= fake[['text', 'class']]

In [ ]:

#  we have 'real' and 'fake' DataFrames
data = pd.concat([real, fake], ignore_index=True)


In [ ]:
data.sample(5)

In [ ]:
# https://github.com/laxmimerit/preprocess_kgptalkie

In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
import preprocess_kgptalkie as ps

data['text'] = data['text'].apply(lambda x: ps.remove_special_chars(x))


In [ ]:
ps.remove_special_chars('this ,. @ # is gre@t')

# Vectoriztion
### convert text data into numeric data


In [ ]:
!pip install gensim
import gensim


In [ ]:
y = data['class'].values


In [ ]:
X=[d.split() for d in data['text'].tolist()]

In [ ]:
type(X[0])

In [ ]:
print(X[0])

In [ ]:
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=X, vector_size=DIM, window=10, min_count=1)


In [ ]:
# Get the number of unique words in the vocabulary
num_unique_words = len(w2v_model.wv.index_to_key)
print(num_unique_words)

In [ ]:
w2v_model.wv['india']


In [ ]:
w2v_model.wv.most_similar('india')


# Train the Machine learing  model
1 direct use vector
2  we can feed these vector as a initial weight in ml model 
then ml will recreate these vectors

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)


In [ ]:
X = tokenizer.texts_to_sequences(X)


In [ ]:
tokenizer.word_index

## Anayze the text data

In [ ]:
[len(x) for x in X]

In [ ]:
plt.hist([len(x) for x in X], bins=700)
plt.show()

In [ ]:
import numpy as np
sequence_lengths = np.array([len(x) for x in X])

# Count the number of sequences with length > 1000
count_greater_than_1000 = len(sequence_lengths[sequence_lengths > 1000])

# Print the count
count_greater_than_1000

In [ ]:
maxlen = 1000  # Define the maximum length you want

# Pad sequences to the defined maximum length
x = pad_sequences(X, maxlen=maxlen)


In [ ]:
len(X[101])

In [ ]:
num_unique_words = len(tokenizer.word_index)
word_index_dict = tokenizer.word_index


In [ ]:
import numpy as np

def get_weight_matrix(model, word_index, num_unique_words, embedding_dim):
    weight_matrix = np.zeros((num_unique_words, embedding_dim))
    for word, i in word_index.items():
        if i < num_unique_words:
            try:
                weight_matrix[i] = model.wv[word]
            except KeyError:
                # Word not found in the Word2Vec model, you can handle this case as needed
                pass
    return weight_matrix


In [ ]:
embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index, num_unique_words, DIM)


In [ ]:
embedding_vectors.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()

# Add an embedding layer with pre-trained Word2Vec embeddings
model.add(Embedding(input_dim=num_unique_words, output_dim=DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))

# Add an LSTM layer with 128 units
model.add(LSTM(units=128))

# Add a dense output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model.fit(X_train, y_train, validation_split=0.3, epochs=6)

In [ ]:
y_pred = (model.predict(X_test) >=0.5).astype(int)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

## Test the ML model

In [ ]:
X_test

In [ ]:
x= ['this is news']
x= tokenizer.texts_to_sequnses(x)
x=pad_sequences(x, maxlen=maxlen)


In [ ]:
(model.predict(x) >=0.5).astype(int)